In [52]:
import pandas as pd 
import time
import requests
import json

API_KEY = "295202-66cdaf"
AIRPORT_IATA = "SLC"
DATE_FROM = "2024-05-10"
DATE_TO = "2024-06-01"

In [53]:
def get_flight_data(airport_iata, date_from, date_to, api_key):
    url = f"https://aviation-edge.com/v2/public/flightsHistory?key={api_key}&code={airport_iata}&type=departure&date_from={date_from}&date_to={date_to}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [54]:
def process_flight_data(flight_data):
    flight_list = []

    for flight in flight_data:
        try:
            flight_list.append({
                "Flight Number": flight.get("flight", {}).get("iataNumber", "N/A"),
                "Airline": flight.get("airline", {}).get("name", "N/A"),
                "Departure Airport": flight.get("departure", {}).get("iataCode", "N/A"),
                "Scheduled Departure": flight.get("departure", {}).get("scheduledTime", "N/A"),
                "Actual Departure": flight.get("departure", {}).get("actualTime", "N/A"),
                "Status": flight.get("status", "N/A"),
                "Delay (minutes)": flight.get("departure", {}).get("delay", "N/A")
            })
        except Exception as e:
            print(f"Error processing flight: {e}")

    return pd.DataFrame(flight_list)



In [ ]:
df = get_flight_data(AIRPORT_IATA, DATE_FROM, DATE_TO, API_KEY)
df = pd.DataFrame(df) 


[{'type': 'departure', 'status': 'active', 'departure': {'iataCode': 'slc', 'icaoCode': 'kslc', 'gate': 'b17', 'delay': 26, 'scheduledTime': '2024-05-09t18:00:00.000', 'estimatedTime': '2024-05-09t18:21:00.000', 'actualTime': '2024-05-09t18:26:00.000', 'estimatedRunway': '2024-05-09t18:26:00.000', 'actualRunway': '2024-05-09t18:26:00.000'}, 'arrival': {'iataCode': 'phx', 'icaoCode': 'kphx', 'terminal': '4', 'baggage': '2', 'gate': 'c6', 'scheduledTime': '2024-05-09t18:45:00.000', 'estimatedTime': '2024-05-09t18:40:00.000'}, 'airline': {'name': 'southwest airlines', 'iataCode': 'wn', 'icaoCode': 'swa'}, 'flight': {'number': '3885', 'iataNumber': 'wn3885', 'icaoNumber': 'swa3885'}}, {'type': 'departure', 'status': 'active', 'departure': {'iataCode': 'slc', 'icaoCode': 'kslc', 'gate': 'b20', 'delay': 13, 'scheduledTime': '2024-05-09t18:20:00.000', 'estimatedTime': '2024-05-09t18:38:00.000', 'actualTime': '2024-05-09t18:32:00.000', 'estimatedRunway': '2024-05-09t18:32:00.000', 'actualRunwa

In [61]:
departure_data = df['departure']

# Convert departure column data into a DataFrame (if it's in dictionary format)
departure_df = pd.DataFrame(departure_data.tolist())

# Extract relevant columns
departure_df_result = departure_df[['delay', 'scheduledTime', 'actualTime']]

# Convert time columns to datetime format
departure_df_result['scheduledTime'] = pd.to_datetime(departure_df_result['scheduledTime'], format='%Y-%m-%dT%H:%M:%S.%f')
departure_df_result['actualTime'] = pd.to_datetime(departure_df_result['actualTime'], format='%Y-%m-%dT%H:%M:%S.%f')

# Extract date and day of the week from 'scheduledTime'
departure_df_result['date'] = departure_df_result['scheduledTime'].dt.date
departure_df_result['Day of the week'] = departure_df_result['scheduledTime'].dt.day_name()

# Display the final DataFrame with the extracted columns
print(departure_df_result)

       delay       scheduledTime          actualTime        date  \
0       26.0 2024-05-09 18:00:00 2024-05-09 18:26:00  2024-05-09   
1       13.0 2024-05-09 18:20:00 2024-05-09 18:32:00  2024-05-09   
2       75.0 2024-05-09 18:30:00                 NaT  2024-05-09   
3       75.0 2024-05-09 18:30:00                 NaT  2024-05-09   
4       75.0 2024-05-09 18:30:00                 NaT  2024-05-09   
...      ...                 ...                 ...         ...   
21606    NaN 2024-06-01 12:50:00 2024-06-01 12:50:00  2024-06-01   
21607    NaN 2024-06-01 17:00:00                 NaT  2024-06-01   
21608    NaN 2024-06-01 16:30:00 2024-06-01 16:26:00  2024-06-01   
21609   10.0 2024-06-01 17:08:00                 NaT  2024-06-01   
21610   14.0 2024-06-01 17:18:00                 NaT  2024-06-01   

      Day of the week  
0            Thursday  
1            Thursday  
2            Thursday  
3            Thursday  
4            Thursday  
...               ...  
21606        Sa

C:\Users\User\AppData\Local\Temp\ipykernel_2884\750878279.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  departure_df_result['scheduledTime'] = pd.to_datetime(departure_df_result['scheduledTime'], format='%Y-%m-%dT%H:%M:%S.%f')
C:\Users\User\AppData\Local\Temp\ipykernel_2884\750878279.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  departure_df_result['actualTime'] = pd.to_datetime(departure_df_result['actualTime'], format='%Y-%m-%dT%H:%M:%S.%f')
C:\Users\User\AppData\Local\Temp\ipykernel_2884\75

In [62]:
airline_data = df['airline']

airline_df = pd.DataFrame(airline_data.tolist())
airline_df_result = airline_df[['name']]
airline_df_result.columns = ['airline']
df['airline'] = airline_df_result['airline']

In [63]:
df[['delay', 'scheduledTime', 'actualTime', 'date', 'Day of the week']] = departure_df_result
df[['airline']] = airline_df_result

df = df.drop(columns = ['codeshared', 'flight', 'status', 'departure', 'arrival'])
df = df[df['airline'].isin(['delta air lines', 'american airlines', 'southwest airlines', 'jetblue airways', 'frontier airlines', 'alaska airlines', 'aeromexico', 'spirit airlines', 'hawaiian airlines'])]

In [65]:
df = pd.DataFrame(df) 
df.to_csv("flights_data.csv")